## Imports

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Data preparation
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from imblearn.over_sampling import SMOTE

/home/paf/miniconda3/lib/python3.6/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/home/paf/miniconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
#Old REPD model
from REPD_Impl import REPD
from autoencoder import AutoEncoder

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
#Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [5]:
#Performance metrics
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import sklearn.metrics as metrics
from sklearn.metrics import f1_score

In [6]:
#Result presentation
from tabulate import tabulate

In [7]:
#Visualization
from matplotlib import pyplot as plt

In [8]:
#Other
import warnings

## Load Dataset support

In [9]:
def load_arff(dataset_name, data_preparation_function):
    #Load data
    data, _ = arff.loadarff("./data/"+dataset+".arff")
    
     # Wrap data into a pandas dataframe
    df = pd.DataFrame(data)
    
    #Prepare data
    df = data_preparation_function(df)
    
    #Return dataframe
    return df

def load_csv(dataset_name, data_preparation_function):
    #Load data
    data = pd.read_csv("./data/"+dataset_name+".csv")
    
    #Prepare data
    df = data_preparation_function(data)
    
    #Return dataframe
    return df

def load_data(dataset_name, dataset_settings):
    if dataset_settings["type"] == "arff":
        return load_arff(dataset_name, dataset_settings["prep_func"])
    elif dataset_settings["type"] == "csv":
        return load_csv(dataset_name, dataset_settings["prep_func"])
    pass

In [10]:
def get_defect_0_1_prep_func(defect_column_name, mapping_function):
    #
    def defect_0_1_prep_func(df):
        df[defect_column_name] = df[defect_column_name].map(mapping_function)
        return df
    #
    return defect_0_1_prep_func


In [11]:
def get_cleanup_prep_func_decorator(prep_func):
    #
    def decorated_prep_func_1(df):
        df = prep_func(df)
        
        #Remove all with missing values
        df = df.dropna()

        #Remove duplicate instances
        df = df.drop_duplicates()
        #
        return df
    #
    return decorated_prep_func_1

In [12]:
def get_adjust_defect_column_name_prep_func_decorator(defecive_column_name, prep_func):
    #
    def decorated_prep_func_2(df):
        df = prep_func(df)
        
        #Rename column
        df = df.rename(columns={defecive_column_name: "defective"})
        #
        return df
    #
    return decorated_prep_func_2        

In [13]:
def get_remove_column_prep_func_decorator(columns_to_remove, prep_func):
    def decorated_prep_func_3(df):
        df = prep_func(df)
        
        #Drop columns
        df = df.drop(columns=columns_to_remove)
        #
        return df
    #
    return decorated_prep_func_3

## Model

In [14]:
class REPD_EX:
    
    def __init__(self, base_repd, defective_classification_model, non_defective_classification_model, use_def_m=True, use_non_def_m=True):
        self.base_repd = base_repd
        self.defective_classification_model = defective_classification_model
        self.non_defective_classification_model = non_defective_classification_model
        self.use_def_m=use_def_m
        self.use_non_def_m=use_non_def_m
    
    def fit(self, X, y, train_base=True):
        #
        if train_base:
            X_m1_fit, X_m2_fit, y_m1_fit, y_m2_fit = train_test_split(X, y, test_size=0.5)
            #
            oversample = SMOTE()
            X_m1_fit, y_m1_fit = oversample.fit_resample(X_m1_fit, y_m1_fit)
            #
            self.base_repd.fit(X_m1_fit,y_m1_fit)
        else:
            X_m2_fit = X
            y_m2_fit = y
        #
        y_p = self.base_repd.predict(X_m2_fit)
        X_r = pd.DataFrame(self.base_repd.transform(X_m2_fit))
        #
        #========================================
        if self.use_def_m:
            X_s_o_t = X_m2_fit[y_p==1]
            X_s_r_t = X_r[y_p==1]
            #
            X_s_o_t = pd.DataFrame(X_s_o_t.values)
            X_s_r_t = pd.DataFrame(X_s_r_t.values)
            #
            X_s_t = pd.concat([X_s_o_t, X_s_r_t], axis=1, join="inner")
            #
            y_s_t = y_m2_fit[y_p==1]
            #
            #print("P Defective:",len(y_s_t[y_s_t==1]),"/",len(y_s_t))
            #
            oversample = SMOTE()
            X_s_t, y_s_t = oversample.fit_resample(X_s_t, y_s_t)
            #
            self.defective_classification_model.fit(X_s_t, y_s_t)
        #========================================
        if self.use_non_def_m:
            X_s_o_t = X_m2_fit[y_p==0]
            X_s_r_t = X_r[y_p==0]
            #
            X_s_o_t = pd.DataFrame(X_s_o_t.values)
            X_s_r_t = pd.DataFrame(X_s_r_t.values)
            #
            X_s_t = pd.concat([X_s_o_t, X_s_r_t], axis=1, join="inner")
            #
            y_s_t = y_m2_fit[y_p==0]
            #
            #print("P Defective:",len(y_s_t[y_s_t==1]),"/",len(y_s_t))
            #
            oversample = SMOTE()
            X_s_t, y_s_t = oversample.fit_resample(X_s_t, y_s_t)
            #
            self.non_defective_classification_model.fit(X_s_t, y_s_t)
        #========================================
        return self
    
    def predict(self, X):
        y_p = self.base_repd.predict(X)
        #
        X_r = pd.DataFrame(self.base_repd.transform(X))
        #
        #========================================
        if self.use_def_m:
            X_s_o_t = X[y_p==1]
            X_s_r_t = X_r[y_p==1]
            #
            X_s_o_t = pd.DataFrame(X_s_o_t.values)
            X_s_r_t = pd.DataFrame(X_s_r_t.values)
            #
            X_s_t = pd.concat([X_s_o_t, X_s_r_t], axis=1, join="inner")
            #
            y_s_p_p = self.defective_classification_model.predict(X_s_t)
        #========================================
        if self.use_non_def_m:
            X_s_o_t = X[y_p==0]
            X_s_r_t = X_r[y_p==0]
            #
            X_s_o_t = pd.DataFrame(X_s_o_t.values)
            X_s_r_t = pd.DataFrame(X_s_r_t.values)
            #
            X_s_t = pd.concat([X_s_o_t, X_s_r_t], axis=1, join="inner")
            #
            y_s_n_p = self.non_defective_classification_model.predict(X_s_t)
        #========================================
        r = []
        #
        cnt1 = 0
        k1 = 0
        cnt2 = 0
        k2 = 0
        for v in y_p:
            if v == 0:
                if self.use_non_def_m:
                    if hasattr(y_s_n_p[k2], "__len__"):
                        nmr = y_s_n_p[k2][0]
                    else:
                        nmr = y_s_n_p[k2]
                    if nmr >= 0.5:
                        r.append(1)
                    else:
                        r.append(0)
                        cnt2 = cnt2 + 1
                    k2 = k2 + 1
                else:
                    r.append(0)
            else:
                if self.use_def_m:
                    if hasattr(y_s_p_p[k1], "__len__"):
                        nmr = y_s_p_p[k1][0]
                    else:
                        nmr = y_s_p_p[k1]
                    if nmr >= 0.5:
                        r.append(1)
                        cnt1 = cnt1 + 1
                    else:
                        r.append(0)
                    k1 = k1 + 1
                else:
                    r.append(1)
        #
        #print("S:", len(X), "P:", len(y_s_p_p), "C1:", cnt1, "N:", len(y_s_n_p), "C2:", cnt2)
        #
        return y_p, np.asarray(r)

## Calculate performance data

In [15]:
def calculate_results(y_true,y_predicted):
    accuracy = accuracy_score(y_true, y_predicted)
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_predicted, average='binary')
    return accuracy, precision, recall, f1_score

## Visualize results

In [16]:
def plot_res(column):
    bp_dict = results_df.boxplot(
        column=column,
        by=["Model"],
        layout=(1,1),       
        return_type='both',
        patch_artist = True,
        vert=False,
    )    
    plt.suptitle("")
    plt.show()

## Experiment

In [17]:
source_datasets = [
    "JDT_R2_0","JDT_R2_1","JDT_R3_0","JDT_R3_1","JDT_R3_2"
]
target_datasets = [
    "PDE_R2_0","PDE_R2_1","PDE_R3_0","PDE_R3_1","PDE_R3_2"    
]

In [18]:
dataset_settings = {
  "JDT_R2_0": {
      "type":"csv",
      "prep_func": get_cleanup_prep_func_decorator(
                      get_remove_column_prep_func_decorator( ["File"],
                      get_adjust_defect_column_name_prep_func_decorator("bug_cnt",
                          get_defect_0_1_prep_func("bug_cnt", lambda x: 1 if x>0 else 0)
                    ))
                  )
  },
  "JDT_R2_1": {
      "type":"csv",
      "prep_func": get_cleanup_prep_func_decorator(
                      get_remove_column_prep_func_decorator( ["File"],
                      get_adjust_defect_column_name_prep_func_decorator("bug_cnt",
                          get_defect_0_1_prep_func("bug_cnt", lambda x: 1 if x>0 else 0)
                    ))
                  )      
  },
  "JDT_R3_0": {
      "type":"csv",
      "prep_func": get_cleanup_prep_func_decorator(
                      get_remove_column_prep_func_decorator( ["File"],
                          get_adjust_defect_column_name_prep_func_decorator("bug_cnt",
                              get_defect_0_1_prep_func("bug_cnt", lambda x: 1 if x>0 else 0)
                      )
                    )
                  )      
  },
  "JDT_R3_1": {
      "type":"csv",
      "prep_func": get_cleanup_prep_func_decorator(
                      get_remove_column_prep_func_decorator( ["File"],
                          get_adjust_defect_column_name_prep_func_decorator("bug_cnt",
                              get_defect_0_1_prep_func("bug_cnt", lambda x: 1 if x>0 else 0)
                      )
                    )
                  )
      
  },
  "JDT_R3_2": {
      "type":"csv",
      "prep_func": get_cleanup_prep_func_decorator(
                      get_remove_column_prep_func_decorator( ["File"],
                          get_adjust_defect_column_name_prep_func_decorator("bug_cnt",
                              get_defect_0_1_prep_func("bug_cnt", lambda x: 1 if x>0 else 0)
                        )
                    )
                  )      
  },
  "PDE_R2_0": {
      "type":"csv",
      "prep_func": get_cleanup_prep_func_decorator(
                      get_remove_column_prep_func_decorator( ["File"],
                      get_adjust_defect_column_name_prep_func_decorator("bug_cnt",
                          get_defect_0_1_prep_func("bug_cnt", lambda x: 1 if x>0 else 0)
                    ))
                  )
  },
  "PDE_R2_1": {
      "type":"csv",
      "prep_func": get_cleanup_prep_func_decorator(
                      get_remove_column_prep_func_decorator( ["File"],
                      get_adjust_defect_column_name_prep_func_decorator("bug_cnt",
                          get_defect_0_1_prep_func("bug_cnt", lambda x: 1 if x>0 else 0)
                    ))
                  )      
  },
  "PDE_R3_0": {
      "type":"csv",
      "prep_func": get_cleanup_prep_func_decorator(
                      get_remove_column_prep_func_decorator( ["File"],
                          get_adjust_defect_column_name_prep_func_decorator("bug_cnt",
                              get_defect_0_1_prep_func("bug_cnt", lambda x: 1 if x>0 else 0)
                      )
                    )
                  )      
  },
  "PDE_R3_1": {
      "type":"csv",
      "prep_func": get_cleanup_prep_func_decorator(
                      get_remove_column_prep_func_decorator( ["File"],
                          get_adjust_defect_column_name_prep_func_decorator("bug_cnt",
                              get_defect_0_1_prep_func("bug_cnt", lambda x: 1 if x>0 else 0)
                      )
                    )
                  )
      
  },
  "PDE_R3_2": {
      "type":"csv",
      "prep_func": get_cleanup_prep_func_decorator(
                      get_remove_column_prep_func_decorator( ["File"],
                          get_adjust_defect_column_name_prep_func_decorator("bug_cnt",
                              get_defect_0_1_prep_func("bug_cnt", lambda x: 1 if x>0 else 0)
                        )
                    )
                  )      
  }
}

In [19]:
dataset_data = {
    
}

In [20]:
for datasets, datasets_type in [(source_datasets,"source"),(target_datasets,"target")]:
    for dataset in datasets:
        print("Loading dataset: ", dataset)
        df = load_data(dataset,dataset_settings[dataset])
        #
        dataset_data[dataset] = {
            "df": df,
            "X": df.drop(columns=["defective"]),
            "y": df["defective"],
            "share": (len(df[df["defective"]==1])/ len(df)),
            "type": datasets_type
        }
        #
        print("Defective:", len(df[df["defective"]==1]), "/", len(df),"=",(len(df[df["defective"]==1])/ len(df)))
        print()

Loading dataset:  JDT_R2_0
Defective: 1069 / 2265 = 0.47196467991169977

Loading dataset:  JDT_R2_1
Defective: 868 / 2591 = 0.3350057892705519

Loading dataset:  JDT_R3_0
Defective: 1310 / 3254 = 0.4025814382298709

Loading dataset:  JDT_R3_1
Defective: 1258 / 3727 = 0.3375368929433861

Loading dataset:  JDT_R3_2
Defective: 814 / 2117 = 0.38450637694851203

Loading dataset:  PDE_R2_0
Defective: 111 / 564 = 0.19680851063829788

Loading dataset:  PDE_R2_1
Defective: 124 / 739 = 0.16779431664411368

Loading dataset:  PDE_R3_0
Defective: 272 / 857 = 0.31738623103850644

Loading dataset:  PDE_R3_1
Defective: 355 / 1059 = 0.3352219074598678

Loading dataset:  PDE_R3_2
Defective: 620 / 1304 = 0.4754601226993865



In [21]:
REPETITION_COUNT = 100
TEST_SIZE = 0.5

In [22]:
for dataset in dataset_data:
    dataset_data[dataset]["train"] = []
    dataset_data[dataset]["test"] = []
    # 
    X = dataset_data[dataset]["X"]
    y = dataset_data[dataset]["y"]
    #
    for _ in range(REPETITION_COUNT):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE)
        #
        dataset_data[dataset]["train"].append((X_train, y_train))
        dataset_data[dataset]["test"].append((X_test, y_test))

In [23]:
def normalize_train(X_train):
    train_min = X_train.min()
    train_dim = X_train.max() - X_train.min()
    train_dim[train_dim == 0] = 1
    #
    X_train = (X_train - train_min) / train_dim
    #
    X_train = X_train.replace([np.inf, -np.inf], np.nan).fillna(0)
    #
    return X_train, train_min, train_dim

def normalize_test(X_test, train_min, train_dim):
    X_test = (X_test - train_min) / train_dim
    #
    X_test = X_test.replace([np.inf, -np.inf], np.nan).fillna(0)
    #
    return X_test

In [24]:
warnings.simplefilter("ignore")

In [25]:
autoencoder = AutoEncoder([48,24],0.01,100,50)
for mode, cls_w_c1, cls_w_c2, use_c1, use_c2 in [
                                ("balance",{0:1., 1: 2.},{0:2., 1: 1.},True, True), 
                                ("hp",{0:1.2, 1: 1.},{0:1., 1: 1.},True,False)
                                ]:
    for source in dataset_data:
        #
        if dataset_data[source]["type"] != "source":
            continue
        #
        X_train = dataset_data[source]["X"]
        y_train = dataset_data[source]["y"]
        #
        X_train, train_min, train_dim = normalize_train(X_train)
        #
        for target in dataset_data:
            if target == source:
                continue
            #
            if dataset_data[target]["type"] != "target":
                continue
            #
            try:
                X_test = dataset_data[target]["X"]
                y_test = dataset_data[target]["y"]
                X_test = normalize_test(X_test, train_min, train_dim)
            except:
                print("Error while preparing data")
                continue            
            #
            print("Mode:",mode," Source:",source," Target:",target," Share:", dataset_data[target]["share"])
            #
            performance_data = []
            #
            for repd_ratio in [0.8,0.7,0.6,0.5,0.4,0.3,0.2]:
                for i in range(REPETITION_COUNT):
                    if (i+1)%2 ==0:
                        print("Repetition: ", (i+1))
                    #============================================================
                    try:
                        X_m1_fit, X_m2_fit, y_m1_fit, y_m2_fit = train_test_split(X_train, y_train, test_size=(1-repd_ratio))
                        #============================================================
                        #REPD
                        print("REPD")
                        autoencoder.re_init()
                        classifer = REPD(autoencoder)
                        classifer.fit(X_m1_fit, y_m1_fit)
                        y_p = classifer.predict(X_test)
                        accuracy, precision, recall, f1_score = calculate_results(y_test, y_p)

                        #Store results
                        data = ['REPD', accuracy, precision, recall, f1_score, source, target, repd_ratio]
                        performance_data.append(data)

                        #REPD_EX
                        print("REPDX")
                        final_classifier_1 = LogisticRegression(class_weight=cls_w_c1)
                        #
                        final_classifier_2 =  LogisticRegression(class_weight=cls_w_c2)
                        #
                        classifer = REPD_EX(base_repd=classifer, 
                                            defective_classification_model=final_classifier_1,
                                            non_defective_classification_model=final_classifier_2,
                                            use_def_m=use_c1,
                                            use_non_def_m=use_c2)
                        classifer.fit(X_m2_fit, y_m2_fit, train_base=False)
                        #
                        y_pb, y_p = classifer.predict(X_test)
                        accuracy_base, precision_base, recall_base, f1_base_score = calculate_results(y_test, y_pb)
                        accuracy, precision, recall, f1_score = calculate_results(y_test, y_p)

                        #Store results
                        data = ['REPD_EX', accuracy, precision, recall, f1_score, source, target, repd_ratio]
                        performance_data.append(data)
                    except:
                        print("Error while running REPD and REPDX")
                        continue
            results_df = pd.DataFrame(performance_data, columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1 score', 'Source', 'Target', 'REPDShare'])
            results_df.to_csv("results_share/cross_"+mode+"_"+source+"_"+target)
            #
            print("Median:", results_df.groupby(["Model","REPDShare"])["F1 score", 'Precision', 'Recall'].median())    
            print()
            print()
autoencoder.close()                    

Mode: balance  Source: JDT_R2_0  Target: PDE_R2_0  Share: 0.19680851063829788
REPD
REPDX
Repetition:  2
REPD
REPDX
REPD
REPDX
Repetition:  4
REPD
REPDX
REPD
REPDX
Repetition:  6
REPD
REPDX
REPD
REPDX
Repetition:  8
REPD
REPDX
REPD
REPDX
Repetition:  10
REPD
REPDX
REPD
REPDX
Repetition:  12
REPD
REPDX
REPD
REPDX
Repetition:  14
REPD
REPDX
REPD
REPDX
Repetition:  16
REPD
REPDX
REPD
REPDX
Repetition:  18
REPD
REPDX
REPD
REPDX
Repetition:  20
REPD
REPDX
REPD
REPDX
Repetition:  22
REPD
REPDX
REPD
REPDX
Repetition:  24
REPD
REPDX
REPD
REPDX
Repetition:  26
REPD
REPDX
REPD
REPDX
Repetition:  28
REPD
REPDX
REPD
REPDX
Repetition:  30
REPD
REPDX
REPD
REPDX
Repetition:  2
REPD
REPDX
REPD
REPDX
Repetition:  4
REPD
REPDX
REPD
REPDX
Repetition:  6
REPD
REPDX
REPD
REPDX
Repetition:  8
REPD
REPDX
REPD
REPDX
Repetition:  10
REPD
REPDX
REPD
REPDX
Repetition:  12
REPD
REPDX
REPD
REPDX
Repetition:  14
REPD
REPDX
REPD
REPDX
Repetition:  16
REPD
REPDX
REPD
REPDX
Repetition:  18
REPD
REPDX
REPD
REPDX
Repetit

REPD
REPDX
Repetition:  26
REPD
REPDX
REPD
REPDX
Repetition:  28
REPD
REPDX
REPD
REPDX
Repetition:  30
REPD
REPDX
REPD
REPDX
Repetition:  2
REPD
REPDX
REPD
REPDX
Repetition:  4
REPD
REPDX
REPD
REPDX
Repetition:  6
REPD
REPDX
REPD
REPDX
Repetition:  8
REPD
REPDX
REPD
REPDX
Repetition:  10
REPD
REPDX
REPD
REPDX
Repetition:  12
REPD
REPDX
REPD
REPDX
Repetition:  14
REPD
REPDX
REPD
REPDX
Repetition:  16
REPD
REPDX
REPD
REPDX
Repetition:  18
REPD
REPDX
REPD
REPDX
Repetition:  20
REPD
REPDX
REPD
REPDX
Repetition:  22
REPD
REPDX
REPD
REPDX
Repetition:  24
REPD
REPDX
REPD
REPDX
Repetition:  26
REPD
REPDX
REPD
REPDX
Repetition:  28
REPD
REPDX
REPD
REPDX
Repetition:  30
REPD
REPDX
Median:                    F1 score  Precision    Recall
Model   REPDShare                               
REPD    0.2        0.388170   0.274323  0.685484
        0.3        0.383502   0.266146  0.681452
        0.4        0.386234   0.268685  0.693548
        0.5        0.385183   0.265592  0.701613
        0.6       

REPD
REPDX
Repetition:  8
REPD
REPDX
REPD
REPDX
Repetition:  10
REPD
REPDX
REPD
REPDX
Repetition:  12
REPD
REPDX
REPD
REPDX
Repetition:  14
REPD
REPDX
REPD
REPDX
Repetition:  16
REPD
REPDX
REPD
REPDX
Repetition:  18
REPD
REPDX
REPD
REPDX
Repetition:  20
REPD
REPDX
REPD
REPDX
Repetition:  22
REPD
REPDX
REPD
REPDX
Repetition:  24
REPD
REPDX
REPD
REPDX
Repetition:  26
REPD
REPDX
REPD
REPDX
Repetition:  28
REPD
REPDX
REPD
REPDX
Repetition:  30
REPD
REPDX
REPD
REPDX
Repetition:  2
REPD
REPDX
REPD
REPDX
Repetition:  4
REPD
REPDX
REPD
REPDX
Repetition:  6
REPD
REPDX
REPD
REPDX
Repetition:  8
REPD
REPDX
REPD
REPDX
Repetition:  10
REPD
REPDX
REPD
REPDX
Repetition:  12
REPD
REPDX
REPD
REPDX
Repetition:  14
REPD
REPDX
REPD
REPDX
Repetition:  16
REPD
REPDX
REPD
REPDX
Repetition:  18
REPD
REPDX
REPD
REPDX
Repetition:  20
REPD
REPDX
REPD
REPDX
Repetition:  22
REPD
REPDX
REPD
REPDX
Repetition:  24
REPD
REPDX
REPD
REPDX
Repetition:  26
REPD
REPDX
REPD
REPDX
Repetition:  28
REPD
REPDX
REPD
REPDX
Repeti

REPD
REPDX
Repetition:  20
REPD
REPDX
REPD
REPDX
Repetition:  22
REPD
REPDX
REPD
REPDX
Repetition:  24
REPD
REPDX
REPD
REPDX
Repetition:  26
REPD
REPDX
REPD
REPDX
Repetition:  28
REPD
REPDX
REPD
REPDX
Repetition:  30
REPD
REPDX
REPD
REPDX
Repetition:  2
REPD
REPDX
REPD
REPDX
Repetition:  4
REPD
REPDX
REPD
REPDX
Repetition:  6
REPD
REPDX
REPD
REPDX
Repetition:  8
REPD
REPDX
REPD
REPDX
Repetition:  10
REPD
REPDX
REPD
REPDX
Repetition:  12
REPD
REPDX
REPD
REPDX
Repetition:  14
REPD
REPDX
REPD
REPDX
Repetition:  16
REPD
REPDX
REPD
REPDX
Repetition:  18
REPD
REPDX
REPD
REPDX
Repetition:  20
REPD
REPDX
REPD
REPDX
Repetition:  22
REPD
REPDX
REPD
REPDX
Repetition:  24
REPD
REPDX
REPD
REPDX
Repetition:  26
REPD
REPDX
REPD
REPDX
Repetition:  28
REPD
REPDX
REPD
REPDX
Repetition:  30
REPD
REPDX
REPD
REPDX
Repetition:  2
REPD
REPDX
REPD
REPDX
Repetition:  4
REPD
REPDX
REPD
REPDX
Repetition:  6
REPD
REPDX
REPD
REPDX
Repetition:  8
REPD
REPDX
REPD
REPDX
Repetition:  10
REPD
REPDX
REPD
REPDX
Repetitio

REPDX
Repetition:  18
REPD
REPDX
REPD
REPDX
Repetition:  20
REPD
REPDX
REPD
REPDX
Repetition:  22
REPD
REPDX
REPD
REPDX
Repetition:  24
REPD
REPDX
REPD
REPDX
Repetition:  26
REPD
REPDX
REPD
REPDX
Repetition:  28
REPD
REPDX
REPD
REPDX
Repetition:  30
REPD
REPDX
Median:                    F1 score  Precision    Recall
Model   REPDShare                               
REPD    0.2        0.383958   0.267309  0.701613
        0.3        0.381803   0.277661  0.616935
        0.4        0.378096   0.272727  0.616935
        0.5        0.378409   0.266655  0.641129
        0.6        0.380353   0.281545  0.604839
        0.7        0.371379   0.274943  0.584677
        0.8        0.379126   0.280857  0.608871
REPD_EX 0.2        0.377477   0.267606  0.645161
        0.3        0.381061   0.269812  0.637097
        0.4        0.385825   0.274908  0.645161
        0.5        0.381528   0.273239  0.649194
        0.6        0.390047   0.276114  0.637097
        0.7        0.399512   0.285714  0.641

...

REPDX
REPD
REPDX
Repetition:  26
REPD
REPDX
REPD
REPDX
Repetition:  28
REPD
REPDX
REPD
REPDX
Repetition:  30
REPD
REPDX
Median:                    F1 score  Precision    Recall
Model   REPDShare                               
REPD    0.2        0.540203   0.484960  0.617647
        0.3        0.539170   0.496361  0.593750
        0.4        0.548590   0.495541  0.621324
        0.5        0.555732   0.500000  0.612132
        0.6        0.558459   0.522213  0.606618
        0.7        0.554544   0.517618  0.602941
        0.8        0.552029   0.518776  0.588235
REPD_EX 0.2        0.397280   0.753505  0.268382
        0.3        0.389187   0.746186  0.268382
        0.4        0.386108   0.757659  0.255515
        0.5        0.359773   0.772727  0.233456
        0.6        0.372356   0.754690  0.242647
        0.7        0.372705   0.753902  0.246324
        0.8        0.350551   0.755111  0.237132


Mode: hp  Source: JDT_R3_1  Target: PDE_R3_1  Share: 0.3352219074598678
REPD
REPDX
Rep

REPDX
Repetition:  8
REPD
REPDX
REPD
REPDX
Repetition:  10
REPD
REPDX
REPD
REPDX
Repetition:  12
REPD
REPDX
REPD
REPDX
Repetition:  14
REPD
REPDX
REPD
REPDX
Repetition:  16
REPD
REPDX
REPD
REPDX
Repetition:  18
REPD
REPDX
REPD
REPDX
Repetition:  20
REPD
REPDX
REPD
REPDX
Repetition:  22
REPD
REPDX
REPD
REPDX
Repetition:  24
REPD
REPDX
REPD
REPDX
Repetition:  26
REPD
REPDX
REPD
REPDX
Repetition:  28
REPD
REPDX
REPD
REPDX
Repetition:  30
REPD
REPDX
REPD
REPDX
Repetition:  2
REPD
REPDX
REPD
REPDX
Repetition:  4
REPD
REPDX
REPD
REPDX
Repetition:  6
REPD
REPDX
REPD
REPDX
Repetition:  8
REPD
REPDX
REPD
REPDX
Repetition:  10
REPD
REPDX
REPD
REPDX
Repetition:  12
REPD
REPDX
REPD
REPDX
Repetition:  14
REPD
REPDX
REPD
REPDX
Repetition:  16
REPD
REPDX
REPD
REPDX
Repetition:  18
REPD
REPDX
REPD
REPDX
Repetition:  20
REPD
REPDX
REPD
REPDX
Repetition:  22
REPD
REPDX
REPD
REPDX
Repetition:  24
REPD
REPDX
REPD
REPDX
Repetition:  26
REPD
REPDX
REPD
REPDX
Repetition:  28
REPD
REPDX
REPD
REPDX
Repetition:

REPDX
REPD
REPDX
Repetition:  22
REPD
REPDX
REPD
REPDX
Repetition:  24
REPD
REPDX
REPD
REPDX
Repetition:  26
REPD
REPDX
REPD
REPDX
Repetition:  28
REPD
REPDX
REPD
REPDX
Repetition:  30
REPD
REPDX
REPD
REPDX
Repetition:  2
REPD
REPDX
REPD
REPDX
Repetition:  4
REPD
REPDX
REPD
REPDX
Repetition:  6
REPD
REPDX
REPD
REPDX
Repetition:  8
REPD
REPDX
REPD
REPDX
Repetition:  10
REPD
REPDX
REPD
REPDX
Repetition:  12
REPD
REPDX
REPD
REPDX
Repetition:  14
REPD
REPDX
REPD
REPDX
Repetition:  16
REPD
REPDX
REPD
REPDX
Repetition:  18
REPD
REPDX
REPD
REPDX
Repetition:  20
REPD
REPDX
REPD
REPDX
Repetition:  22
REPD
REPDX
REPD
REPDX
Repetition:  24
REPD
REPDX
REPD
REPDX
Repetition:  26
REPD
REPDX
REPD
REPDX
Repetition:  28
REPD
REPDX
REPD
REPDX
Repetition:  30
REPD
REPDX
REPD
REPDX
Repetition:  2
REPD
REPDX
REPD
REPDX
Repetition:  4
REPD
REPDX
REPD
REPDX
Repetition:  6
REPD
REPDX
REPD
REPDX
Repetition:  8
REPD
REPDX
REPD
REPDX
Repetition:  10
REPD
REPDX
REPD
REPDX
Repetition:  12
REPD
REPDX
REPD
REPDX
Rep

Mode: hp  Source: JDT_R3_2  Target: PDE_R3_1  Share: 0.3352219074598678
REPD
REPDX
Repetition:  2
REPD
REPDX
REPD
REPDX
Repetition:  4
REPD
REPDX
REPD
REPDX
Repetition:  6
REPD
REPDX
REPD
REPDX
Repetition:  8
REPD
REPDX
REPD
REPDX
Repetition:  10
REPD
REPDX
REPD
REPDX
Repetition:  12
REPD
REPDX
REPD
REPDX
Repetition:  14
REPD
REPDX
REPD
REPDX
Repetition:  16
REPD
REPDX
REPD
REPDX
Repetition:  18
REPD
REPDX
REPD
REPDX
Repetition:  20
REPD
REPDX
REPD
REPDX
Repetition:  22
REPD
REPDX
REPD
REPDX
Repetition:  24
REPD
REPDX
REPD
REPDX
Repetition:  26
REPD
REPDX
REPD
REPDX
Repetition:  28
REPD
REPDX
REPD
REPDX
Repetition:  30
REPD
REPDX
REPD
REPDX
Repetition:  2
REPD
REPDX
REPD
REPDX
Repetition:  4
REPD
REPDX
REPD
REPDX
Repetition:  6
REPD
REPDX
REPD
REPDX
Repetition:  8
REPD
REPDX
REPD
REPDX
Repetition:  10
REPD
REPDX
REPD
REPDX
Repetition:  12
REPD
REPDX
REPD
REPDX
Repetition:  14
REPD
REPDX
REPD
REPDX
Repetition:  16
REPD
REPDX
REPD
REPDX
Repetition:  18
REPD
REPDX
REPD
REPDX
Repetition:  

REPD
REPDX
REPD
REPDX
Repetition:  28
REPD
REPDX
REPD
REPDX
Repetition:  30
REPD
REPDX
REPD
REPDX
Repetition:  2
REPD
REPDX
REPD
REPDX
Repetition:  4
REPD
REPDX
REPD
REPDX
Repetition:  6
REPD
REPDX
REPD
REPDX
Repetition:  8
REPD
REPDX
REPD
REPDX
Repetition:  10
REPD
REPDX
REPD
REPDX
Repetition:  12
REPD
REPDX
REPD
REPDX
Repetition:  14
REPD
REPDX
REPD
REPDX
Repetition:  16
REPD
REPDX
REPD
REPDX
Repetition:  18
REPD
REPDX
REPD
REPDX
Repetition:  20
REPD
REPDX
REPD
REPDX
Repetition:  22
REPD
REPDX
REPD
REPDX
Repetition:  24
REPD
REPDX
REPD
REPDX
Repetition:  26
REPD
REPDX
REPD
REPDX
Repetition:  28
REPD
REPDX
REPD
REPDX
Repetition:  30
REPD
REPDX
Median:                    F1 score  Precision    Recall
Model   REPDShare                               
REPD    0.2        0.514801   0.590521  0.456452
        0.3        0.512814   0.597177  0.451613
        0.4        0.505820   0.594063  0.441129
        0.5        0.507741   0.592716  0.452419
        0.6        0.506130   0.592111  0.433